# BioSimSpace - Calculating Relative Free Energies

BioSimSpace can bring together all the tools needed to calculate relative free energies.

Here, we will do a small example, calculating the relative hydration free energy of ethane and methanol.


In [ ]:
import BioSimSpace as BSS

We start by constructing both molecules from smiles strings (using RDKit) and parameterising (using GAFF in Antechamber).

In [ ]:
ethane = BSS.Parameters.gaff("CC").getMolecule()
methanol = BSS.Parameters.gaff("CO").getMolecule()

Relative free energy calculations are computational alchemy! We will transform our reference molecule (ethane) into the perturbed molecule (methanol) and will calculate the change in free energy during this transformation.

First, we have to map the atoms of the reference molecule onto the perturbed molecule.

In [ ]:
mapping = BSS.Align.matchAtoms(ethane, methanol)

BioSimSpace provides a neat visualisation of this mapping via py3dmol

In [ ]:
BSS.Align.viewMapping(ethane, methanol, mapping)

Using this mapping, we now need to align methanol on top of ethane, such that mapped atoms are (almost) directly on top of each other.

In [ ]:
ethane = BSS.Align.rmsdAlign(ethane, methanol, mapping)

We are now ready to merge these two molecules together. This will create a hybrid molecule which contains the parameters and coordinates of both. This hybrid merged molecule can be transformed using a λ-coordinate.

The aim is that the merged molecule looks like ethane at λ=0, and it looks like methanol at λ=1.

In [ ]:
merged = BSS.Align.merge(ethane, methanol, mapping)

Here is the merged molecule at λ=0

In [ ]:
view = BSS.Notebook.View(merged.molecule0())
view.system()

And here it is at λ=1

In [ ]:
view = BSS.Notebook.View(merged.molecule1())
view.system()

We now need to solvate this molecule in water, just as we did before. This uses gmx solvate from GROMACS under the hood.

In [ ]:
solvated = BSS.Solvent.tip3p(molecule=merged, box=3 * [5 * BSS.Units.Length.nanometer])

Now we create the simulation protocol. This time we will use a free energy protocol. This protocol runs multiple molecular dynamics simulations, one for each λ-value. Here we will use four λ-values, running 5 ps of dynamics in each window.

In [ ]:
protocol = BSS.Protocol.FreeEnergyProduction(num_lam=4, runtime=5*BSS.Units.Time.picosecond)

Just like before, we can set up a simulation by combining a system and a protocol. Here we combine the system with the free energy protocol, passing it to the `FreeEnergy.Relative` driver. This adapts the system and protocol for whatever underlying engine is on your system. Here we choose GROMACS.

In [ ]:
free_gmx = BSS.FreeEnergy.Relative(solvated, protocol, engine="gromacs", work_dir="freenrg_gmx/free")

This creates the `freenrg_gmx/free` directory, in which there is a simulation directory for each λ-value.

In [ ]:
!ls freenrg_gmx/free

In [ ]:
!ls freenrg_gmx/free/lambda_0.0000

We need to run two relative free energy calculations - one for the perturbation in water, and one in vacuum. Here we set up the input files needed for the vacuum calculation. 

In [ ]:
vac_gmx = BSS.FreeEnergy.Relative(merged.toSystem(), protocol, engine="gromacs", work_dir="freenrg_gmx/vacuum")

The above set up the files for GROMACS. You could take these and submit them to a cluster to run. But we have the same code to create the input files for other free energy engines, e.g. here we create the input files for `somd`.

In [ ]:
free_somd = BSS.FreeEnergy.Relative(solvated, protocol, engine="somd", work_dir="freenrg_somd/free")

In [ ]:
vac_somd = BSS.FreeEnergy.Relative(merged.toSystem(), protocol, engine="somd", work_dir="freenrg_somd/vacuum")

In [ ]:
!ls freenrg_somd/vacuum

In [ ]:
!ls freenrg_somd/vacuum/lambda_0.0000

`somd` is GPU-accelerated, so we can run the vacuum simulations live here!

In [ ]:
vac_somd.run()

In [ ]:
vac_somd.wait()

The simulations in the water box take a bit longer (about 10 minutes). We could run them now, but "here's one I prepared earlier" :-)

In [ ]:
# free_somd.run() - this takes about 10 minutes
# free_somd.wait()

In [ ]:
!ls example_somd/free/lambda_0.0000

We analyse the results by passing in the name of the directory that contains the simulations to the `analyse` function. Let's first calculate the free energy of transforming ethane to methanol in vacuum..,

In [ ]:
vac_pmf, overlaps = BSS.FreeEnergy.Relative.analyse("freenrg_somd/vacuum")
vac_pmf

Now we will calculate the free energy of transforming ethane to methanol in water...

In [ ]:
free_pmf, overlaps = BSS.FreeEnergy.Relative.analyse("example_somd/free")
free_pmf

The difference between these two is the relative hydration free energy.

In [ ]:
free_nrg = BSS.FreeEnergy.Relative.difference(free_pmf, vac_pmf)
free_nrg

Not bad, given the experimental value is -6.90 kcal mol-1.